# Librerías

In [2]:
import pandas as pd
import numpy as np
import os

Listado del total de los datos.

In [3]:
os.listdir('../data/')

['product_blocks.csv',
 'products.csv',
 'stock_day_71_stock_replenishment_days_72-77.csv',
 'positions_day_71.csv',
 'positions_day_0_day_63.csv',
 'stock_and_sales_day_0_day_63.csv']

#### Product_blocks

In [4]:
product_blocks = pd.read_csv('../data/product_blocks.csv')

In [5]:
product_blocks.head()

,product_id,block_id
0,612967398,0
1,296892108,0
2,139541214,0
3,963923934,0
4,938230141,0


In [8]:
product_blocks.block_id.value_counts().head()

2506    10
1717    10
2666    10
962     10
407     10
Name: block_id, dtype: int64

#### Products

In [9]:
products = pd.read_csv('../data/products.csv')
products.head()

,product_id,family_id,subfamily_id,price
0,151926,396066037,335531561,25.95
1,213413,552529755,11509337,19.95
2,310130,679611953,533441312,12.95
3,373883,775013441,62518413,17.95
4,455200,998145072,490222156,29.95


#### Stock_day_71_stock_replenishment_days_72-77

In [89]:
day_71_77 = pd.read_csv('../data/stock_day_71_stock_replenishment_days_72-77.csv')
day_71_77.head()

,product_id,color_id,size_id,stock_at_day_71,stock_replenishment_days_72,stock_replenishment_days_73,stock_replenishment_days_74,stock_replenishment_days_75,stock_replenishment_days_76,stock_replenishment_days_77
0,151926,909958217,54666687,73,0,0,2,0,0,1
1,151926,909958217,111455099,16,0,0,0,0,0,0
2,151926,909958217,242364222,111,0,1,-1,1,0,0
3,151926,909958217,513017386,47,0,0,0,1,1,1
4,151926,909958217,820445075,71,0,1,-2,2,0,0


In [96]:
len(day_71_77.product_id.value_counts())

8892

#### Positions_day_71-77

In [11]:
position_71 = pd.read_csv('../data/positions_day_71.csv')
position_71.head()

,date_number,product_id,category_id,position
0,71,65968023,2312968,6
1,71,70320877,2312968,24
2,71,70700940,2312968,22
3,71,76636979,2312968,1
4,71,83181798,2312968,0


In [91]:
position_71.date_number.value_counts()

71    20029
Name: date_number, dtype: int64

In [97]:
len(position_71.product_id.value_counts())

7728

#### Positions_day_0_day_63

In [12]:
position_0_63 = pd.read_csv('../data/positions_day_0_day_63.csv')
position_0_63.head()

,date_number,product_id,category_id,position
0,0,4450020,4461548,17
1,0,42147334,4461548,4
2,0,81131830,4461548,35
3,0,84035833,4461548,38
4,0,125252584,4461548,39


In [93]:
position_0_63.date_number.value_counts().head()

63    18688
61    18472
60    18429
62    18249
59    18244
Name: date_number, dtype: int64

#### Stock_and_sales_day_0_day_63

In [13]:
sales_0_63 = pd.read_csv('../data/stock_and_sales_day_0_day_63.csv')
sales_0_63.head()

,date_number,product_id,color_id,size_id,sales,stock
0,0,310130,78467031,7026791,1,93
1,0,310130,78467031,8015276,2,62
2,0,310130,78467031,452052101,1,18
3,0,310130,78467031,541898038,2,42
4,0,310130,78467031,558685710,3,113


# Generación de los conjuntos TRAIN / TEST

El primer objetivo de la competición es formar un conjunto de entrenamiento de modelos y otro de test.

Para comenzar, iremos mergeando los diferentes datasets hasta dar con un primer conjunto de datos inicial. Uniremos los datasets de ventas-stock y posicionamiento en la web para los días de 0 a 63

In [75]:
df_0_63 = position_0_63.join(sales_0_63.set_index(['date_number', 'product_id']),
                             on=['date_number', 'product_id'],
                             how='inner')
df_0_63.head()

,date_number,product_id,category_id,position,color_id,size_id,sales,stock
0,0,4450020,4461548,17,10926303,242364222,1,1
0,0,4450020,4461548,17,10926303,513017386,4,11
0,0,4450020,4461548,17,51733619,242364222,6,168
0,0,4450020,4461548,17,51733619,513017386,0,66
0,0,4450020,4461548,17,51733619,820445075,3,119


In [76]:
print(sales_0_63.shape, position_0_63.shape, df_0_63.shape)

(2421061, 6) (969897, 4) (5420282, 8)


Nulls

In [78]:
df_0_63.isnull().sum().sum()

0

In [79]:
nan_rows = df_0_63[df_0_63.isnull().T.any().T]
nan_rows.head()

,date_number,product_id,category_id,position,color_id,size_id,sales,stock


In [82]:
df_0_63_products = df_0_63.join(products.set_index(['product_id']), on=['product_id'], how='inner')
df_0_63_products.head()

,date_number,product_id,category_id,position,color_id,size_id,sales,stock,family_id,subfamily_id,price
0,0,4450020,4461548,17,10926303,242364222,1,1,511255811,731664934,12.95
0,0,4450020,4461548,17,10926303,513017386,4,11,511255811,731664934,12.95
0,0,4450020,4461548,17,51733619,242364222,6,168,511255811,731664934,12.95
0,0,4450020,4461548,17,51733619,513017386,0,66,511255811,731664934,12.95
0,0,4450020,4461548,17,51733619,820445075,3,119,511255811,731664934,12.95


In [83]:
print(df_0_63_products.shape, df_0_63.shape, products.shape)

(5420282, 11) (5420282, 8) (15238, 4)


In [88]:
df_0_63_products_blocks = df_0_63_products.join(product_blocks.set_index(['product_id']), on=['product_id'], how='inner')
df_0_63_products_blocks.head(20)

,date_number,product_id,category_id,position,color_id,size_id,sales,stock,family_id,subfamily_id,price,block_id
0,0,4450020,4461548,17,10926303,242364222,1,1,511255811,731664934,12.95,1779
0,0,4450020,4461548,17,10926303,513017386,4,11,511255811,731664934,12.95,1779
0,0,4450020,4461548,17,51733619,242364222,6,168,511255811,731664934,12.95,1779
0,0,4450020,4461548,17,51733619,513017386,0,66,511255811,731664934,12.95,1779
0,0,4450020,4461548,17,51733619,820445075,3,119,511255811,731664934,12.95,1779
0,0,4450020,4461548,17,329416505,242364222,3,115,511255811,731664934,12.95,1779
0,0,4450020,4461548,17,329416505,513017386,1,48,511255811,731664934,12.95,1779
0,0,4450020,4461548,17,329416505,820445075,0,107,511255811,731664934,12.95,1779
0,0,4450020,4461548,17,735224030,242364222,6,162,511255811,731664934,12.95,1779
0,0,4450020,4461548,17,735224030,513017386,0,52,511255811,731664934,12.95,1779


In [86]:
print(df_0_63_products_blocks.shape, df_0_63_products.shape, product_blocks.shape)

(5420282, 12) (5420282, 11) (15238, 2)


In [87]:
df_0_63_products_blocks[df_0_63_products_blocks.isnull().T.any().T].shape

(0, 12)

Con esto que hemos hecho tendríamos el conjunto de entrenamiento. Ahora tenemos q sacar los "diferentes" conjuntos de validación.

